In [23]:
!pip install fastai
!pip install imgaug

     |████████████████████████████████| 948 kB 473 kB/s eta 0:00:01     |█████████████████████████▎      | 747 kB 473 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 3.9 MB/s eta 0:00:01     |██                              | 204 kB 3.9 MB/s eta 0:00:01     |██▋                             | 266 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 4.3 MB/s eta 0:00:01    |██▌                             | 931 kB 5.0 MB/s eta 0:00:03


     |████████████████████████████████| 1.6 MB 8.3 MB/s eta 0:00:01     |████████▎                       | 419 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 2.9 MB/s eta 0:00:01


In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%reload_ext autoreload

In [3]:
import os
import numpy as np
from pathlib import Path
from argparse import Namespace

args = Namespace(
    root = "/".join(os.getcwd().split("/")[:-1]),
    bucket_name = "sagemaker-us-west-2-430127992102",
    key = "TiendaApp",
    image_dir = "fastai_data",
    data_dir = "input/data/training",
    tar_file_name = "fastai_data.tar.gz",
    model_dir = "model",
    model_fastai_dir = "fastai_model",
    tar_model_name = "fastai_model.tar.gz"
)

In [4]:
data_path = Path(os.path.join(args.root, args.data_dir))
data_path
os.listdir(data_path)

['yogurt_alpina_finesse_fresax180g',
 'coca_cola_originalx400ml',
 'coca_cola_originalx250ml',
 'arroz_basmati_castellanox1000g',
 'chicharron_americano_jacksx15g',
 'papas_limon_margaritax110g',
 'arroz_vitamor_dianax500g',
 '.DS_Store',
 'papas_limon_margaritax39g',
 'mango_tommy',
 'yogurt_alpina_finesse_fresax150g',
 'papas_limon_margaritax105g',
 'models',
 'yogurt_alpina_finesse_frutos_rojosx180g',
 'pan_mantequilla_la_fronterax55g',
 'papas_limon_margarita_packeconomicox300g',
 'maracuya',
 'aguacate',
 'yogurt_alpina_finesse_melocotonx180g',
 'spagheti_barilla',
 'arroz_blanco_carullax1000g',
 'pan_de_queso_la_frontera']

In [1]:
import fastai
print(fastai.__version__)

2.0.0


In [13]:
!pip install fastcore==1.0.0

In [2]:
import fastcore
fastcore.__version__

'0.1.39'

In [23]:
from fastai2.vision.all import *

data = ImageDataLoaders.from_folder(data_path, train='.', valid_pct=0.2, batch_tfms=16,
            item_tfms=Resize(224), seed=21, num_workers=4, no_check=True)

Could not do one pass in your dataloader, there is something wrong in it


In [24]:
data.show_batch(figsize=(9,8))

TypeError: 'int' object is not callable

In [20]:
learn = cnn_learner(data, resnet34, metrics=error_rate)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [22]:
learn.fit_one_cycle(4)

RuntimeError: DataLoader worker (pid 893) is killed by signal: Killed. 

In [21]:
learn.fine_tune(1)

RuntimeError: DataLoader worker (pid 877) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.